In [229]:
import pandas as pd
import numpy as np
import geopandas as gpd
from bokeh.plotting import figure, save, show
import os
# IMPORTANT NOTE: having difficulting writing the crs to shp using geopands to_file
# Can do it manually using notepad++ though. WORK IN PROGRESS
from shapely.geometry import Point
import matplotlib.pyplot as plt
%matplotlib inline
import janmaps as jm # personal functions for mapping
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper
import pysal as ps
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper

# Create the color mapper
color_mapper = LogColorMapper(palette=palette)

import plotly
import plotly.plotly as py
plotly.tools.set_credentials_file(username='XXX',api_key='XXX')
from plotly.graph_objs import *
plotly.offline.init_notebook_mode(connected=True)

In [230]:
p = figure(title = 'My first interactive plot')
AWU = gpd.read_file(r'D:\Documents\ML\Geo\shapefiles\ServiceArea\AWUServiceArea.shp')
zpco = gpd.read_file(r'D:\Documents\ML\Geo\shapefiles\zipcodes\Austinzipcodes.shp')
df = pd.read_excel(r'D:\Documents\ML\Geo\Leaks_forCo.xlsx',sheet_name='Sheet1')

In [231]:
coordX = df.X
coordY = df.Y

# Create geodataframe from data
pts_geometry = [Point(xy) for xy in zip(coordX, coordY)]
pts_crs = {'init':'epsg:3857'}
points = gpd.GeoDataFrame(df, crs=pts_crs, geometry=pts_geometry)

In [232]:
zpco = zpco.to_crs(pts_crs)
AWU = AWU.to_crs(pts_crs)
z_geo = zpco.geometry
AWU_geo = AWU.geometry
p_geo = points.geometry

In [233]:
cut_boundary = AWU.loc[0].geometry
new_area = zpco[zpco.geometry.intersects(cut_boundary)]

In [268]:
points

,HomeNo,SubAddress1,StName,SubAddress2,SubAddress3,City,ZIP,AddressID,MapZone,ObjectID,...,DaysElapsed,Week,Month,Year,Bi-Week,DateOnly,FullStName,geometry,x,y
0,9605,N,FM 973,RD,NaN,AUSTIN,78653,3374522,NaN,1038936,...,NaN,40,9,2016,40,9/29/2016,9605 N FM 973 RD,POINT (-10860848.8235 3542441.27329999),-1.086085e+07,-1.086085e+07
1,7304,NaN,LONGFORD,TRL,NaN,AUSTIN,78653,5092550,528G,1056782,...,0.317464,42,10,2016,42,10/14/2016,7304 LONGFORD TRL,POINT (-10865000.6851 3549474.4441),-1.086500e+07,-1.086500e+07
2,6413,NaN,BOYCE,LN,NaN,MANOR,78653,3047359,528B,1048143,...,0.126284,41,10,2016,42,10/7/2016,6413 BOYCE LN,POINT (-10865699.4999 3550194.7531),-1.086570e+07,-1.086570e+07
3,6413,NaN,BOYCE,LN,NaN,MANOR,78653,3047359,528B,1050326,...,1.053158,41,10,2016,42,10/8/2016,6413 BOYCE LN,POINT (-10865699.4999 3550194.7531),-1.086570e+07,-1.086570e+07
4,5600,NaN,WAR ADMIRAL,DR,NaN,DEL VALLE,78617,3049059,678P,1097471,...,0.273281,49,11,2016,50,11/28/2016,5600 WAR ADMIRAL DR,POINT (-10865812.9825 3525692.1875),-1.086581e+07,-1.086581e+07
5,9775,E,US 290 HWY,SVRD,2018-01-02 00:00:00,AUSTIN,78724,5077135,528X,1261617,...,0.476818,17,4,2017,18,4/26/2017,9775 1/2 E US 290 HWY SVRD EB,POINT (-10865815.5598 3546548.6322),-1.086582e+07,-1.086582e+07
6,16118,NaN,FM 812,RD,NaN,AUSTIN,78617,3372550,738P,1213275,...,0.407314,10,3,2017,10,3/10/2017,16118 FM 812 RD,POINT (-10865883.3765 3516941.32859999),-1.086588e+07,-1.086588e+07
7,9500,NaN,DECKER,LN,NaN,AUSTIN,78724,3050731,NaN,1169608,...,NaN,4,1,2017,4,1/27/2017,9500 DECKER LN,POINT (-10865906.9717 3544933.22439999),-1.086591e+07,-1.086591e+07
8,15001,NaN,JACOBSON,RD,NaN,DEL VALLE,78617,99534,NaN,1312146,...,NaN,28,7,2017,28,7/10/2017,15001 JACOBSON RD,POINT (-10865926.1574 3520651.5318),-1.086593e+07,-1.086593e+07
9,11920,NaN,DUNFRIES,LN,NaN,AUSTIN,78754,3127585,528F,1033915,...,0.295520,39,9,2016,40,9/24/2016,11920 DUNFRIES LN,POINT (-10865932.9761 3549424.0451),-1.086593e+07,-1.086593e+07


In [235]:
leaks_per_zip = points.groupby(['ZIP'])['geometry'].count()

In [252]:
new_area[['zipcode']]=new_area[['zipcode']].astype(int)

C:\Users\ASUS\Anaconda3\lib\site-packages\pandas\core\frame.py:2352: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [237]:
leaks_per_zip = pd.DataFrame(leaks_per_zip)

In [238]:
lpz = leaks_per_zip.reset_index()

In [239]:
lpz.columns = (['zipcode','leaks'])

In [263]:
df_join = pd.merge(new_area,lpz,how='left',left_on='zipcode',right_on='zipcode')

In [265]:
df_join['leaks'] = df_join['leaks'].fillna(0)
df_join.sample(10)

,objectid,name,shape_len,zipcodes_i,modified_d,created_da,modified_b,created_by,geodb_oid,shape_area,zipcode,geometry,leaks
2,5.0,AUSTIN,121428.596913,58.0,None,None,None,None,5.0,4.792590e+08,78732,POLYGON ((-10895175.64269098 3559984.048974039...,2.0
8,20.0,AUSTIN,103966.108075,14.0,None,None,None,None,20.0,3.345571e+08,78733,POLYGON ((-10891827.27213824 3545149.136407329...,0.0
34,55.0,MANCHACA,133031.079976,49.0,None,None,None,None,55.0,3.194486e+08,78652,POLYGON ((-10898634.42507883 3524724.843856914...,8.0
39,8.0,AUSTIN,65759.003768,2.0,None,None,None,None,8.0,2.696128e+08,78729,POLYGON ((-10878294.07237547 3564570.652051724...,103.0
54,68.0,AUSTIN,34474.871995,79.0,None,None,None,None,68.0,4.756859e+07,78701,"POLYGON ((-10879920.4168834 3535856.191094069,...",51.0
5,15.0,AUSTIN,55142.876610,9.0,None,None,None,None,15.0,1.953556e+08,78723,POLYGON ((-10872062.18614475 3540319.176226617...,155.0
19,38.0,AUSTIN,175315.944319,32.0,None,None,None,None,38.0,5.590181e+08,78725,POLYGON ((-10867385.13009736 3537606.506163087...,11.0
3,6.0,AUSTIN,206132.564319,59.0,None,None,None,None,6.0,1.320054e+09,78737,POLYGON ((-10911397.99422128 3534651.607400042...,17.0
55,69.0,ROUND ROCK,158356.379338,80.0,None,None,None,None,69.0,9.765688e+08,78664,"POLYGON ((-10864918.55703228 3572692.25638406,...",1.0
21,40.0,AUSTIN,62429.872755,34.0,None,None,None,None,40.0,2.151014e+08,78741,POLYGON ((-10876640.64976244 3535756.802076177...,163.0


In [256]:
leaks = df_join['leaks']

In [257]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [258]:
def getPointCoords(row, geom, coord_type):
    """Calculates coordinates ('x' or 'y') of a Point geometry"""
    if coord_type == 'x':
        return row[geom].x
    elif coord_type == 'y':
        return row[geom].y

In [259]:
points['x'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
points['y'] = points.apply(getPointCoords, geom='geometry', coord_type='x', axis=1)
p_df = points.drop('geometry',axis=1).copy()
psource = ColumnDataSource(p_df)

In [260]:
df_join['x'] = df_join.apply(getPolyCoords, geom = 'geometry', coord_type = 'x', axis =1)
df_join['y'] = df_join.apply(getPolyCoords, geom = 'geometry', coord_type = 'y', axis =1)
z_df = df_join.drop('geometry',axis=1).copy()
zsource = ColumnDataSource(z_df)

In [272]:
p = figure(title="Leaks map")

# Plot grid
patches = p.patches('x', 'y', source=zsource,fill_color={'field': 'leaks', 'transform': color_mapper},
         fill_alpha=1.0, line_color="black", line_width=0.5)

# Add metro on top of the same figure
#p.multi_line('x', 'y', source=msource, color="red", line_width=2)

# Add points on top (as black points)
circle = p.circle('X','Y',source=psource, size=1, color="white")

my_hover1 = HoverTool(renderers = [patches])
my_hover1.tooltips =[('leaks', '@leaks')]

my_hover2 = HoverTool(renderers = [circle])
my_hover2.tooltips =[('location', '@FullStName'),('time leaks','@DateOnly')]

p.add_tools(my_hover1)
p.add_tools(my_hover2)
# Save the figure
outfp = r'D:\Documents\ML\Geo\leaksmap.html'
save(obj = p, filename = outfp)

C:\Users\ASUS\Anaconda3\lib\site-packages\bokeh\io\saving.py:125: UserWarning:

save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN

C:\Users\ASUS\Anaconda3\lib\site-packages\bokeh\io\saving.py:138: UserWarning:

save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'



'D:\\Documents\\ML\\Geo\\leaksmap.html'